### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  

In [1]:
import pandas as pd
import numpy as np

Pymoli_data = "purchase_data.csv"
df = pd.read_csv(Pymoli_data)
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
players = df['SN'].nunique()
players

576

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.
* Create a summary data frame to hold the results
* Display the summary data frame

In [3]:
total_items = df['Item ID'].nunique()
avg_price = df['Price'].mean()
total_purchases = df['Purchase ID'].count()
total_sales = df['Price'].sum()

columns = ['Number of Unique Items', 'Average Price', 'Number of Purchases', 'Total Revenue']
summ = pd.DataFrame({
    "Number of Unique Items": [total_items],
    "Average Price": [avg_price],
    "Number of Purchases": [total_purchases],
    "Total Revenue": [total_sales]
})
summ = summ.style.format({'Average Price': '${:.3}', 'Total Revenue': '${:.6}'})
summ

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [4]:
demo = df.loc[:, ['SN', 'Age', 'Gender']].drop_duplicates()
demo = demo['Gender'].value_counts()
demo_pct = demo/players*100
genDemo_summary = pd.DataFrame({
    "Total Count": demo, 
    "Percentage": demo_pct
}).style.format({'Percentage': '{:.4}%'})
genDemo_summary

,Total Count,Percentage
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
* Create a summary data frame to hold the results
* Display the summary data frame

In [5]:
sales_analysis = df.groupby(["Gender"])["Price"].agg(['sum', 'count', 'mean'])
gen_sales_total = sales_analysis['sum']
avg_individual_sales = gen_sales_total.div(demo)
gen_sales_summary = pd.DataFrame({
                    "Purchase Count": sales_analysis['count'],
                    "Average Purchase Price": sales_analysis['mean'],
                    "Total Sales per Person": sales_analysis['sum'],
                    "Avg Sale per Person": avg_individual_sales
}).style.format({'Average Purchase Price': '${:.3}', 
                    'Total Sales per Person': '${:.6}', 
                    'Avg Sale per Person': '${:.3}'})
gen_sales_summary

,Purchase Count,Average Purchase Price,Total Sales per Person,Avg Sale per Person
Gender,,,,
Female,113,$3.2,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages
* Categorize the existing players using the age bins. Hint: use pd.cut()
* Calculate the numbers and percentages by age group
* Create a summary data frame to hold the results
* Display Age Demographics Table

In [6]:
ages = [0, 9.99, 14.99, 19.99, 24.99, 29.99, 34.99, 39.99, 1000]
binnames = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

age_demo_df = df.loc[:, ["SN", "Age"]].drop_duplicates()
ageDemo = age_demo_df["Age"]
ageDemo["Age"] = pd.cut(ageDemo, ages, labels=binnames)
ageDemo = (pd.DataFrame(ageDemo["Age"].value_counts()))
ageDemo.sort_index(inplace=True)
ageDemo['Percentage'] =  pd.Series(["{0:.2f}%".format((age_count / players) * 100) for age_count in ageDemo["Age"]], index = ageDemo.index)
ageDemo.rename(columns={'Age': 'Age Range Count'}, inplace=True)
ageDemo

,Age Range Count,Percentage
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
* Create a summary data frame to hold the results
* Display the summary data frame

In [7]:
df["Ages"] = pd.cut(df["Age"], ages, labels=binnames)
sales_by_age = df.groupby(["Ages"])["Price"].agg(['sum', 'count', 'mean'])

sales_by_age_summary = pd.DataFrame({
                    "Purchase Count": sales_by_age['count'],
                    "Average Purchase Price": sales_by_age['mean'],
                    "Total Sales per Person": sales_by_age['sum'],
                    "Avg Sales per Person": sales_by_age['sum'] / \
                    ageDemo['Age Range Count']
}).head(5).style.format({'Average Purchase Price': '${:.3}', 
                    'Total Sales per Person': '${:.6}', 
                    'Avg Sales per Person': '${:.3}'})
sales_by_age_summary

,Purchase Count,Average Purchase Price,Total Sales per Person,Avg Sales per Person
Ages,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.9,$293.0,$3.81


## Top Spenders

* Run basic calculations to obtain the results in the table below
* Create a summary data frame to hold the results
* Sort the total purchase value column in descending order
* Display a preview of the summary data frame

In [8]:
indv_user_sales = df.groupby(["SN"])["Price"].agg(['sum', 'count', 'mean'])

indv_user_sales_summary = pd.DataFrame({
                    "Purchase Count": indv_user_sales['count'],
                    "Average Purchase Price": indv_user_sales['mean'],
                    "Total Sales per Person": indv_user_sales['sum']
}).sort_values("Total Sales per Person", ascending=False) \
                    .head(5).style.format({'Average Purchase Price': '${:.3}', 
                    'Total Sales per Person': '${:.6}', 
                    'Avg Sale per Person': '${:.3}'})
indv_user_sales_summary

,Purchase Count,Average Purchase Price,Total Sales per Person
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.4,$13.62
Iskadarya95,3,$4.37,$13.1


### Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns
* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value
* Create a summary data frame to hold the results
* Sort the purchase count column in descending order
* Display a preview of the summary data frame

In [19]:
itemData = df.groupby(['Item ID', 'Item Name'])['Price'].agg(['sum', 'count', 'mean'])
itemData.columns = ["Item Total Sales", "Purchase Count", "Avg Item Price"]
itemData_df = pd.DataFrame(itemData).sort_values("Purchase Count", ascending=False)\
                    .head(20).style.format({'Avg Item Price': '${:.6}', 
                    'Item Total Sales': '${:.4}'})
itemData_df

,,Item Total Sales,Purchase Count,Avg Item Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$50.76,12,$4.23
145,Fiery Glass Crusader,$41.22,9,$4.58
108,"Extraction, Quickblade Of Trembling Hands",$31.77,9,$3.53
82,Nirvana,$44.1,9,$4.9
19,"Pursuit, Cudgel of Necromancy",$8.16,8,$1.02
103,Singed Scalpel,$34.8,8,$4.35
75,Brutality Ivory Warmace,$19.36,8,$2.42
72,Winter's Bite,$30.16,8,$3.77
60,Wolf,$28.32,8,$3.54


## Most Profitable Items

* Display a preview of the data frame

In [11]:
profitable_items = pd.DataFrame(itemData) \
                    .sort_values("Item Total Sales", ascending=False) \
                    .head(5).style.format({'Avg Item Price': '${:.6}', 
                    'Item Total Sales': '${:.4}'})
profitable_items

,,Item Total Sales,Purchase Count,Avg Item Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$50.76,12,$4.23
82,Nirvana,$44.1,9,$4.9
145,Fiery Glass Crusader,$41.22,9,$4.58
92,Final Critic,$39.04,8,$4.88
103,Singed Scalpel,$34.8,8,$4.35


## Trends

-Contrary to intuition, the age bracket that spends the most per transaction is the <10 years old bracket.
-Male players spend less than ~10% less average relative to females non-disclosed/others.
-The cost of the item does not appear to influence the frequency of purchase.  In fact, the more expensive items tend to be frequently bought than cheaper items.